# IST 736 Text Mining

### NewsApi.org NFL Player news scrape

David Madsen (dfmadsen@syr.edu)
11/30/2019

## 1. Objective

Load a list of teams and rosters. Cycle through each teams roster and query newsapi.org for player headlines in a seven day timespan.

In [53]:
# import packages
import datetime
import json
import os
import pandas as pd
import re
import requests

In [23]:
# load the team and roster data
teams_df = pd.read_csv('../text_mine_stats/nfl_teams.tsv', sep = '\t', encoding='utf-8')
teams_df

,Unnamed: 0,team,year,roster,text
0,0,new_england_patriots,2019,"['Martez Ivey', 'Brandon Bolden', 'Jarrett Sti...",{'team_roster_news': {'2019-11-30T17:55:03.429...
1,1,oakland_raiders,2019,"['Maxx Crosby', 'Makinton Dorleant', 'Benson M...",{'team_roster_news': {'2019-11-30T17:55:04.352...
2,2,washington_redskins,2019,"['Morgan Moses', 'Shaun Dion Hamilton', 'Monta...",{'team_roster_news': {'2019-11-30T17:55:05.530...
3,3,kansas_city_chiefs,2019,"['Alex Officer', 'Joe Fortson', 'Derrick Nnadi...",{'team_roster_news': {'2019-11-30T17:55:06.676...
4,4,tampa_bay_buccaneers,2019,"['Evan Perroni', 'John Battle', 'Jordan Whiteh...",{'team_roster_news': {'2019-11-30T17:55:07.610...
5,5,new_orleans_saints,2019,"['Eldridge Massington', 'Terron Armstead', 'Te...",{'team_roster_news': {'2019-11-30T17:55:08.707...
6,6,philadelphia_eagles,2019,"['Alshon Jeffery', 'Rick Lovato', 'Mack Hollin...",{'team_roster_news': {'2019-11-30T17:55:09.447...
7,7,dallas_cowboys,2019,"['Deante Burton', 'Christian Covington', 'Zack...",{'team_roster_news': {'2019-11-30T17:55:10.188...
8,8,los_angeles_chargers,2019,"['Roderic Teamer Jr.', 'Mike Williams', 'Ty Mc...",{'team_roster_news': {'2019-11-30T17:55:11.287...
9,9,detroit_lions,2019,"['Casey Tucker', 'Jamal Agnew', ""Da'shawn Hand...",{'team_roster_news': {'2019-11-30T17:55:12.039...


In [24]:
teams_df.head(3)

,Unnamed: 0,team,year,roster,text
0,0,new_england_patriots,2019,"['Martez Ivey', 'Brandon Bolden', 'Jarrett Sti...",{'team_roster_news': {'2019-11-30T17:55:03.429...
1,1,oakland_raiders,2019,"['Maxx Crosby', 'Makinton Dorleant', 'Benson M...",{'team_roster_news': {'2019-11-30T17:55:04.352...
2,2,washington_redskins,2019,"['Morgan Moses', 'Shaun Dion Hamilton', 'Monta...",{'team_roster_news': {'2019-11-30T17:55:05.530...


In [30]:
teams_dict={}
for row in teams_df.itertuples():
    roster = re.sub("\'", "", row[4])
    roster = roster.strip('[')
    roster = roster.strip(']')
    teams_dict[row[2]] = roster.split(',')
    
teams_dict

{'new_england_patriots': ['Martez Ivey',
  ' Brandon Bolden',
  ' Jarrett Stidham',
  ' Jermaine Eluemunor',
  ' Devin McCourty',
  ' Cameron Meredith',
  ' Byron Cowart',
  ' David Andrews',
  ' Duron Harmon',
  ' Calvin Munson',
  ' Quincy Adeboyejo',
  ' Phillip Dorsett',
  ' Korey Cunningham',
  ' Terrence Brooks',
  ' James White',
  ' Obi Melifonwu',
  ' "NKeal Harry"',
  ' Matt LaCosse',
  ' Jakob Johnson',
  ' Shaq Mason',
  ' Stephon Gilmore',
  ' Elandon Roberts',
  ' Julian Edelman',
  ' "Jawhaun Bentley"',
  ' Isaiah Wynn',
  ' John Simon',
  ' James Ferentz',
  ' Ryan Izzo',
  ' Chase Winovich',
  ' J.C. Jackson',
  ' Jakobi Meyers',
  ' Joe Thuney',
  ' Benjamin Watson',
  ' Rex Burkhead',
  ' Tyler Gauthier',
  ' Marcus Cannon',
  ' Cody Kessler',
  ' Marshall Newhouse',
  ' Matthew Slater',
  ' Yodny Cajuste',
  ' Jonathan Jones',
  ' Tom Brady',
  ' Adam Butler',
  ' Nate Ebner',
  ' Tashawn Bower',
  ' Patrick Chung',
  ' Mohamed Sanu',
  ' Kyle Van Noy',
  ' Lawrence

In [34]:
for t in teams_dict.keys():
    roster=teams_dict[t]
    print(roster)
    break;

['Martez Ivey', ' Brandon Bolden', ' Jarrett Stidham', ' Jermaine Eluemunor', ' Devin McCourty', ' Cameron Meredith', ' Byron Cowart', ' David Andrews', ' Duron Harmon', ' Calvin Munson', ' Quincy Adeboyejo', ' Phillip Dorsett', ' Korey Cunningham', ' Terrence Brooks', ' James White', ' Obi Melifonwu', ' "NKeal Harry"', ' Matt LaCosse', ' Jakob Johnson', ' Shaq Mason', ' Stephon Gilmore', ' Elandon Roberts', ' Julian Edelman', ' "Jawhaun Bentley"', ' Isaiah Wynn', ' John Simon', ' James Ferentz', ' Ryan Izzo', ' Chase Winovich', ' J.C. Jackson', ' Jakobi Meyers', ' Joe Thuney', ' Benjamin Watson', ' Rex Burkhead', ' Tyler Gauthier', ' Marcus Cannon', ' Cody Kessler', ' Marshall Newhouse', ' Matthew Slater', ' Yodny Cajuste', ' Jonathan Jones', ' Tom Brady', ' Adam Butler', ' Nate Ebner', ' Tashawn Bower', ' Patrick Chung', ' Mohamed Sanu', ' Kyle Van Noy', ' Lawrence Guy', ' Jason McCourty', ' Brian Schwenke', ' Shilique Calhoun', ' Jamie Collins', ' Terez Hall', ' Keionta Davis', ' Da

In [46]:
# set up some query parameters for NewsApi.org
base_url = 'https://newsapi.org/v2/'
controller = 'everything'
current_time = datetime.datetime.now()
last_week = current_time - datetime.timedelta(days=7)

# put query times into the correct format for one week's query 
current_time = current_time.isoformat()
last_week = last_week.isoformat()

In [61]:
# load an API key for NewsApi.org
with open('../text_mine_sentiment_players/newsapi_credentials.json') as cred_file:
    newsapi_creds = json.load(cred_file)
    newsapi_key = newsapi_creds['API_KEY']

In [63]:
# set up the URL parameters
url_params = {'apiKey':newsapi_key,
             'from':last_week,
             'to':current_time,
             'language':'en',
             'pageSize':100}

In [ ]:
player_articles = []
completed_teams = ['new_england_patriots', 'oakland_raiders']

for team in teams_dict.keys():
    if (team in completed_teams):
        continue
    for player in teams_dict[team]:
        url_params['q'] = '"' + player + '"'
        response = requests.get(base_url + controller, url_params)
        jsontxt = response.json()
        if not "articles" in jsontxt.keys():
            print(jsontxt)
        else:
            for article in jsontxt["articles"]:
                article_dict = {'team_name':team,
                               'player_name':player,
                               'title':article['title'],
                               'description':article['description'],
                               'content':article['content']}
                player_articles.append(article_dict)
    


In [72]:
player_articles_df = pd.DataFrame(player_articles)
player_articles_df.to_csv('../text_mine_sentiment_players/player_articles_2.tsv', sep='\t', encoding='utf-8')

In [75]:
player_articles_df.team_name.unique()

array(['new_england_patriots', 'oakland_raiders', 'washington_redskins'],
      dtype=object)